In [1]:
import pandas as pd

DataLoading and Merging

In [2]:
files = ["api_data_aadhar_demographic_0_500000.csv", "api_data_aadhar_demographic_500000_1000000.csv", "api_data_aadhar_demographic_1000000_1500000.csv","api_data_aadhar_demographic_1500000_2000000.csv","api_data_aadhar_demographic_2000000_2071700.csv"]

df_final = pd.concat(
    (pd.read_csv(f) for f in files),
    ignore_index=True
)


In [3]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071700 entries, 0 to 2071699
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   date           object
 1   state          object
 2   district       object
 3   pincode        int64 
 4   demo_age_5_17  int64 
 5   demo_age_17_   int64 
dtypes: int64(3), object(3)
memory usage: 94.8+ MB


Date Data Type Correction

In [4]:
df_final['date'] = pd.to_datetime(
    df_final['date'],
    format='%d-%m-%Y',
    errors='raise'
)


In [5]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071700 entries, 0 to 2071699
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date           datetime64[ns]
 1   state          object        
 2   district       object        
 3   pincode        int64         
 4   demo_age_5_17  int64         
 5   demo_age_17_   int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 94.8+ MB


Addition of Support Column Total Demography and year_month

In [6]:
df_final['Total Demography']=(df_final['demo_age_17_']+df_final['demo_age_5_17'])

In [7]:
df_final['year_month']=df_final['date'].dt.to_period('M').dt.to_timestamp()

In [8]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071700 entries, 0 to 2071699
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   date              datetime64[ns]
 1   state             object        
 2   district          object        
 3   pincode           int64         
 4   demo_age_5_17     int64         
 5   demo_age_17_      int64         
 6   Total Demography  int64         
 7   year_month        datetime64[ns]
dtypes: datetime64[ns](2), int64(4), object(2)
memory usage: 126.4+ MB


State Cleaning

State Feature contains lots of wrongly spelled states so we done basic normalisation and replacement technique to clean this column

In [9]:
df_final['state'].unique()

array(['Uttar Pradesh', 'Andhra Pradesh', 'Gujarat', 'Rajasthan',
       'Karnataka', 'West Bengal', 'Telangana', 'Odisha', 'Maharashtra',
       'Kerala', 'Bihar', 'Tamil Nadu', 'Madhya Pradesh', 'Assam',
       'Tripura', 'Arunachal Pradesh', 'Punjab', 'Jharkhand', 'Delhi',
       'Chandigarh', 'Chhattisgarh', 'Jammu and Kashmir', 'Mizoram',
       'Nagaland', 'Himachal Pradesh', 'Goa', 'Haryana', 'Meghalaya',
       'Uttarakhand', 'Manipur', 'Daman and Diu', 'Puducherry', 'Sikkim',
       'Ladakh', 'Dadra and Nagar Haveli and Daman and Diu',
       'Dadra and Nagar Haveli', 'Orissa', 'Pondicherry',
       'Andaman & Nicobar Islands', 'Andaman and Nicobar Islands',
       'west Bengal', 'Daman & Diu', 'West  Bengal', 'odisha',
       'Jammu & Kashmir', 'Lakshadweep', 'Dadra & Nagar Haveli',
       'Westbengal', 'andhra pradesh', 'WEST BENGAL', 'West Bangal',
       'West bengal', 'ODISHA', 'WESTBENGAL', 'Chhatisgarh',
       'West Bengli', 'Darbhanga', 'Puttenahalli', 'BALANAGAR',
  

In [ ]:
#Removing Special Characters
df_final['state'] = (
    df_final['state']
    .astype(str)
    .str.strip()
    .str.replace(r'\s+', ' ', regex=True)
    .str.title()
)


In [11]:
state_map = {
    # Spelling / case variants
    'Westbengal': 'West Bengal',
    'West Bangal': 'West Bengal',
    'West Bengli': 'West Bengal',

    # Old names
    'Orissa': 'Odisha',
    'Uttaranchal': 'Uttarakhand',
    'Pondicherry': 'Puducherry',
    'Chhatisgarh': 'Chhattisgarh',

    # J&K variants
    'Jammu & Kashmir': 'Jammu and Kashmir',

    # Andaman variants
    'Andaman & Nicobar Islands': 'Andaman and Nicobar Islands',

    # UT merger variants
    'Daman And Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman & Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dadra & Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dadra And Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    '100000':'Unknown'
}


In [12]:
df_final['state'] = df_final['state'].replace(state_map)


In [13]:
invalid_states = {
    'Darbhanga', 'Jaipur', 'Nagpur', 'Puttenahalli',
    'Balanagar', 'Madanapalle', 'Raja Annamalai Puram'
}

df_final.loc[
    df_final['state'].isin(invalid_states),
    'state'
] = 'Unknown'


In [14]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071700 entries, 0 to 2071699
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   date              datetime64[ns]
 1   state             object        
 2   district          object        
 3   pincode           int64         
 4   demo_age_5_17     int64         
 5   demo_age_17_      int64         
 6   Total Demography  int64         
 7   year_month        datetime64[ns]
dtypes: datetime64[ns](2), int64(4), object(2)
memory usage: 126.4+ MB


District Cleaning

Similar to state district have also been cleaned on the same technique

In [15]:
df_final['district'].unique()

array(['Gorakhpur', 'Chittoor', 'Rajkot', 'Srikakulam', 'Udaipur',
       'Sikar', 'Tumakuru', 'Kurnool', 'Paschim Medinipur', 'Ghazipur',
       'Patan', 'Mulugu', 'Ganganagar', 'Nayagarh', 'Nashik',
       'Shivamogga', 'Thrissur', 'Hassan', 'Patna', 'Belgaum',
       'Kancheepuram', 'Jabalpur', 'Chennai', 'Tinsukia', 'Jamui', 'Gaya',
       'Bengaluru', 'Tiruppur', 'Jalgaon', 'Dhalai', 'Nabarangapur',
       'Chittorgarh', 'Darbhanga', 'Purnia', 'Muzaffarnagar', 'Jaipur',
       'Churu', 'Bharatpur', 'Jamnagar', 'Howrah', 'Ahmedabad',
       'West Kameng', 'Jalor', 'Belagavi', 'Bahraich', 'Nagapattinam',
       'Sheikhpura', 'Amritsar', 'Mysore', 'Alluri Sitharama Raju',
       'Sitamarhi', 'Khunti', 'Saharanpur', 'Kanyakumari', 'Prakasam',
       'Ananthapuramu', 'Hooghly', 'Central Delhi', 'Deoria', 'Giridih',
       'Koraput', 'Shahjahanpur', 'Chhotaudepur', 'Jalandhar', 'Kannauj',
       'Visakhapatnam', 'Jodhpur', 'Karimnagar', 'Amethi', 'Hyderabad',
       'Salem', 'Cuddalore'

In [16]:
df_final['district'] = (
    df_final['district']
    .astype(str)
    .str.strip()
    .str.replace(r'\xa0', ' ', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.replace(r'\*', '', regex=True)
    .str.replace(r'\(.*?\)', '', regex=True)
    .str.replace(r'[-–—]', ' ', regex=True)
    .str.title()
)


The District Column contained some unusual pattern and keywords which was normalised to unknown

In [17]:
garbage_patterns = (
    r'Near |Road|Colony|Hospital|Thana|Cross|Dist :|Garden'
)

df_final.loc[
    df_final['district'].str.contains(garbage_patterns, regex=True, na=False),
    'district'
] = 'Unknown'


In [18]:
df_final.loc[
    df_final['district'].isin(['North', 'South', 'East', 'West', 'North East']),
    'district'
] = 'Unknown'


Wrongly Spelled District are normalised using basic technique of replacement

In [19]:
district_map = {
    # Karnataka
    'Belgaum': 'Belagavi',
    'Mysore': 'Mysuru',
    'Gulbarga': 'Kalaburagi',
    'Bellary': 'Ballari',

    # Telangana
    'K V Rangareddy': 'Ranga Reddy',
    'K.V. Rangareddy': 'Ranga Reddy',
    'Rangareddi': 'Ranga Reddy',

    # West Bengal
    'Hooghiy': 'Hooghly',
    'Haora': 'Howrah',

    # Odisha
    'Jajapur': 'Jajpur',
    'Jaipur': 'Jajpur',  # if Odisha Jaipur slipped
    'Sonapur': 'Subarnapur',

    # UP renames
    'Allahabad': 'Prayagraj',
    'Faizabad': 'Ayodhya',
    'Jyotiba Phule Nagar': 'Amroha',

    # Medchal chaos
    'Medchal Malkajgiri': 'Medchal–Malkajgiri',
}


In [20]:
df_final['district'] = df_final['district'].replace(district_map)


In [21]:
df_final.loc[df_final['district'].isna(), 'district'] = 'Unknown'

In [23]:
df_final['district'].unique()

array(['Gorakhpur', 'Chittoor', 'Rajkot', 'Srikakulam', 'Udaipur',
       'Sikar', 'Tumakuru', 'Kurnool', 'Paschim Medinipur', 'Ghazipur',
       'Patan', 'Mulugu', 'Ganganagar', 'Nayagarh', 'Nashik',
       'Shivamogga', 'Thrissur', 'Hassan', 'Patna', 'Belagavi',
       'Kancheepuram', 'Jabalpur', 'Chennai', 'Tinsukia', 'Jamui', 'Gaya',
       'Bengaluru', 'Tiruppur', 'Jalgaon', 'Dhalai', 'Nabarangapur',
       'Chittorgarh', 'Darbhanga', 'Purnia', 'Muzaffarnagar', 'Jajpur',
       'Churu', 'Bharatpur', 'Jamnagar', 'Howrah', 'Ahmedabad',
       'West Kameng', 'Jalor', 'Bahraich', 'Nagapattinam', 'Sheikhpura',
       'Amritsar', 'Mysuru', 'Alluri Sitharama Raju', 'Sitamarhi',
       'Khunti', 'Saharanpur', 'Kanyakumari', 'Prakasam', 'Ananthapuramu',
       'Hooghly', 'Central Delhi', 'Deoria', 'Giridih', 'Koraput',
       'Shahjahanpur', 'Chhotaudepur', 'Jalandhar', 'Kannauj',
       'Visakhapatnam', 'Jodhpur', 'Karimnagar', 'Amethi', 'Hyderabad',
       'Salem', 'Cuddalore', 'Sabarkan

In [24]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071700 entries, 0 to 2071699
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   date              datetime64[ns]
 1   state             object        
 2   district          object        
 3   pincode           int64         
 4   demo_age_5_17     int64         
 5   demo_age_17_      int64         
 6   Total Demography  int64         
 7   year_month        datetime64[ns]
dtypes: datetime64[ns](2), int64(4), object(2)
memory usage: 126.4+ MB


In [26]:
df_final['state'].unique()

array(['Uttar Pradesh', 'Andhra Pradesh', 'Gujarat', 'Rajasthan',
       'Karnataka', 'West Bengal', 'Telangana', 'Odisha', 'Maharashtra',
       'Kerala', 'Bihar', 'Tamil Nadu', 'Madhya Pradesh', 'Assam',
       'Tripura', 'Arunachal Pradesh', 'Punjab', 'Jharkhand', 'Delhi',
       'Chandigarh', 'Chhattisgarh', 'Jammu And Kashmir', 'Mizoram',
       'Nagaland', 'Himachal Pradesh', 'Goa', 'Haryana', 'Meghalaya',
       'Uttarakhand', 'Manipur',
       'Dadra and Nagar Haveli and Daman and Diu', 'Puducherry', 'Sikkim',
       'Ladakh', 'Dadra And Nagar Haveli And Daman And Diu',
       'Andaman and Nicobar Islands', 'Andaman And Nicobar Islands',
       'Jammu and Kashmir', 'Lakshadweep', 'Unknown'], dtype=object)

In [27]:
state_wise_district_count = (
    df_final
    .groupby('state')['district']
    .nunique()
    .reset_index(name='district_count')
    .sort_values('district_count', ascending=False)
)

state_wise_district_count.head(10)


,state,district_count
37,Uttar Pradesh,87
22,Madhya Pradesh,60
23,Maharashtra,51
18,Karnataka,48
39,West Bengal,45
31,Rajasthan,45
5,Bihar,45
33,Tamil Nadu,45
2,Andhra Pradesh,44
34,Telangana,41


In [28]:
df_final.head(50)

,date,state,district,pincode,demo_age_5_17,demo_age_17_,Total Demography,year_month
0,2025-03-01,Uttar Pradesh,Gorakhpur,273213,49,529,578,2025-03-01
1,2025-03-01,Andhra Pradesh,Chittoor,517132,22,375,397,2025-03-01
2,2025-03-01,Gujarat,Rajkot,360006,65,765,830,2025-03-01
3,2025-03-01,Andhra Pradesh,Srikakulam,532484,24,314,338,2025-03-01
4,2025-03-01,Rajasthan,Udaipur,313801,45,785,830,2025-03-01
5,2025-03-01,Rajasthan,Sikar,332028,28,285,313,2025-03-01
6,2025-03-01,Karnataka,Tumakuru,572201,88,332,420,2025-03-01
7,2025-03-01,Uttar Pradesh,Gorakhpur,273211,61,836,897,2025-03-01
8,2025-03-01,Andhra Pradesh,Kurnool,518313,83,986,1069,2025-03-01
9,2025-03-01,West Bengal,Paschim Medinipur,721148,13,281,294,2025-03-01


In [29]:
df_final.to_csv('Demography_cleaned_final.csv',index=False)